In [ ]:
import os
import pandas as pd
import numpy as np

# process data and save into csv
for each measurement configuration (num. of streamed variables, num. of oscillators in filterbank), obtain the average and maximum (worst case) latency, jitter, and average CPU load. 

In [ ]:
# list files in folder 
num_oscs = [0, 20, 40]
num_aux_vars = [1, 5, 10, 20, 50, 100]
configurations = [ [aux_num,osc_num] for osc_num in num_oscs for aux_num in num_aux_vars]

# write header to csv file
with open("data/processed.csv", "a") as f:
    f.write("aux_num,osc_num,mean,jitter,max,cpu_load\n")


for config in configurations:
    
    aux_num, osc_num = config[0], config[1]
    config_array = f"USB-V{aux_num}-O{osc_num}"
    
    # retrieve files from data/folder that contain the configuration name
    files = [f for f in os.listdir("data") if config_array in f]
    # get mean cpu load
    cpu_load_file = [f for f in files if "cpu-load.log" in f][0]
    cpu_load_df = pd.read_csv(f"data/{cpu_load_file}", sep=" ", header=None).dropna().iloc[:, [1]]  # Keep only the second column, which is the cpu_load column    
    # drop last six rows (they show measurements after the Bela code is stopped)
    cpu_load_df = cpu_load_df.iloc[:-6]
    cpu_load_array = cpu_load_df.iloc[:, 0].str.rstrip('%').astype(float).to_numpy() / 100
    # cap values >100 to 100
    cpu_load_array[cpu_load_array > 1] = 1
    mean_cpu_load = np.mean(cpu_load_array)
    
    diff_file = [f for f in files if "diffs.csv" in f][0]
    diff_df = pd.read_csv(f"data/{diff_file}")

    # calc jitter as the difference between the 97.5 and 2.5 percentiles of the mean across all variables
    diff_df_mean_across_vars = diff_df.mean(axis=1)    
    lower_bound = diff_df_mean_across_vars.quantile(0.025)
    upper_bound = diff_df_mean_across_vars.quantile(0.975)
    jitter = upper_bound - lower_bound
    
    # get mean and max across all variables (not using std as it's not a normal distribution)
    stats = diff_df.describe()
    mean = stats.loc["mean"].mean().round(1) # average across variables
    max = stats.loc["max"].max().round(1)

    print(f"{config_array}, mean: {mean:.1f} ms, jitter: {jitter:.1f} ms, max: {max:.1f} ms, cpu_load: {mean_cpu_load:.2}")

    # store in csv file
    with open("data/processed.csv", "a") as f:
        f.write(f"{aux_num},{osc_num},{mean:.1f},{jitter:.1f},{max:.1f},{mean_cpu_load:.2}\n")
    

USB-V1-O0, mean: 7.6 ms, jitter: 10.1 ms, max: 33.0 ms, cpu_load: 0.37
USB-V5-O0, mean: 25.0 ms, jitter: 36.2 ms, max: 122.6 ms, cpu_load: 0.68
USB-V10-O0, mean: 47.8 ms, jitter: 65.9 ms, max: 196.6 ms, cpu_load: 0.67
USB-V20-O0, mean: 95.9 ms, jitter: 120.1 ms, max: 292.1 ms, cpu_load: 0.66
USB-V50-O0, mean: 240.2 ms, jitter: 180.9 ms, max: 602.6 ms, cpu_load: 0.67
USB-V100-O0, mean: 475.9 ms, jitter: 221.7 ms, max: 767.0 ms, cpu_load: 0.67
USB-V1-O20, mean: 12.3 ms, jitter: 23.7 ms, max: 71.8 ms, cpu_load: 0.76
USB-V5-O20, mean: 47.7 ms, jitter: 95.1 ms, max: 221.0 ms, cpu_load: 0.9
USB-V10-O20, mean: 93.6 ms, jitter: 145.7 ms, max: 352.7 ms, cpu_load: 0.9
USB-V20-O20, mean: 186.7 ms, jitter: 254.6 ms, max: 546.0 ms, cpu_load: 0.89
USB-V50-O20, mean: 469.1 ms, jitter: 378.1 ms, max: 989.0 ms, cpu_load: 0.9
USB-V100-O20, mean: 894.9 ms, jitter: 432.6 ms, max: 1347.1 ms, cpu_load: 0.88
USB-V1-O40, mean: 43.0 ms, jitter: 107.7 ms, max: 195.9 ms, cpu_load: 0.98
USB-V5-O40, mean: 193.2 ms

export to latex

In [28]:
df = pd.read_csv("data/processed.csv")
#sort by aux_num and osc_num
df = df.sort_values(by=["aux_num", "osc_num"])

# export to latex
with open('table.tex', 'w') as tf:
     tf.write(df.to_latex())